This notebook is used to process the flight schedule data.
Namely, we will simulate the nnumber of passenger on each flight. We will simply assume that it is normally distributed over a mean of 30 and a standard deviation of 50.

## Set-up

In [1]:
# import packages
import numpy as np 
import math
import pandas as pd 
import os 

/Users/Tra_FIT/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# load the csv files
data_files = os.listdir("./data")

## Processing
We will add a column to each of the flight schedule we have and parse the date as appropriate.
Note that the current data file only includes the time but not the date. 

In [54]:
file = data_files[1]
df = pd.read_csv(f"./data/{file}")

In [55]:
# add a column of integer passenger load per flight
np.random.seed(2024)
df['pass_load'] = np.random.normal(300,50,size =len(df))
df['pass_load']  = df['pass_load'].apply(lambda x: math.floor(x))


In [56]:
df.head()

,code,time_sch,dest,time_act,commercial,orig,pass_load
0,CX 261,00:05,Paris,Dep 00:09,1,HKG,383
1,RH 358,00:05,Almaty\nMilan/MXP,Dep 07:08,0,HKG,336
2,CX 2035,00:10,HKG,At gate 23:55 (20/12/2023),0,Osaka/Kansai\nTaipei,289
3,MU 725,00:15,HKG,At gate 00:29,1,Shanghai/PVG,292
4,CX 289,00:15,Frankfurt,Dep 00:17,1,HKG,345


In [57]:
# parse the schedule time to add the date
month = file[2:5]
date = file[0:2]
time_sch = pd.Series(f"2023 {month} {date} " + df['time_sch'])
df['time_sch'] = pd.to_datetime(time_sch, format = "%Y %b %d %H:%M")

# parse the actual time
# try:
    


In [58]:
df.head()

,code,time_sch,dest,time_act,commercial,orig,pass_load
0,CX 261,2023-12-21 00:05:00,Paris,Dep 00:09,1,HKG,383
1,RH 358,2023-12-21 00:05:00,Almaty\nMilan/MXP,Dep 07:08,0,HKG,336
2,CX 2035,2023-12-21 00:10:00,HKG,At gate 23:55 (20/12/2023),0,Osaka/Kansai\nTaipei,289
3,MU 725,2023-12-21 00:15:00,HKG,At gate 00:29,1,Shanghai/PVG,292
4,CX 289,2023-12-21 00:15:00,Frankfurt,Dep 00:17,1,HKG,345


In [53]:
# if file is on amsterdam
if file[10:13] == "AMS":
    # the actual time either on time or delayed
    time_act = pd.Series(f"2023 {month} {date} " + df['time_act'])
    df['time_act'] = pd.to_datetime(time_act, format = "%Y %b %d %H:%M")
    df["time_diff"]  = df.apply(lambda x:(x['time_act'] - x['time_sch']).total_seconds(), axis = 1)
    # columns with negative time difference
    indices = df.query("time_diff < 0").index
    for ind in indices:
        df.loc[ind,"time_act"] = df.loc[ind,"time_act"] + pd.Timedelta(1, unit = "D")

elif file[10:13] == "HKG":
    # parse the time
    time_act_t = df["time_act"].str.split(":")
    time_act_t = time_act_t.apply(lambda x: f"{x[0][-2:]}:{x[1][:2]}")
    # parse the date
    time_act_d = df["time_act"].str.split("/")
    time_act_d = time_act_d.apply(lambda x: x[0][-2:] if len(x) > 1 else None)
    time_act_d = time_act_d.fillna(date)
    # concatenate the time and the date
    df['time_act'] = "2023 " + month + " " + time_act_d + " " + time_act_t
    df['time_act'] = pd.to_datetime(time_act, format = "%Y %b %d %H:%M")
    

SyntaxError: unexpected EOF while parsing (<ipython-input-53-856f607a48ea>, line 12)